### Library import

In [1]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 200)

### Read in data

In [2]:
items_root = '../data/australian_users_items.json'
items_rows = []
with open(items_root, encoding='MacRoman') as f:
    for line in f.readlines():
        items_rows.append(ast.literal_eval(line))

df_items = pd.DataFrame(items_rows)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### Saving data

In [4]:
users = df_items[['user_id', 'items_count', 'steam_id', 'user_url']]
users.to_csv('../data/csv/users.csv')

In [5]:
# transform user items df to have all the games by id and name
items = {'item_id':[], 'item_name':[]}
for i, row in df_items.iterrows():
    items_list = row['items']
    for item in items_list:
        if item['item_id'] not in items['item_id']:
            items['item_id'].append(item['item_id'])
            items['item_name'].append(item['item_name'])
items_df_transformed = pd.DataFrame(items)
items_df_transformed.head()

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [7]:
games = items_df_transformed.rename(columns={'item_id':'game_id','item_name':'name'})
games.to_csv('../data/csv/games.csv')
games.head()

,game_id,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [17]:
games2 = pd.read_json('../data/output_steam_games.json', lines=True)[['id','app_name']].rename(columns={'id':'game_id','app_name':'name'})
games2.dropna(how='all' ,inplace=True)
games2.dropna(subset='game_id', inplace=True)
games2['game_id'] = games2['game_id'].astype('int64')
games2.head()

,game_id,name
88310,761140,Lost Summoner Kitty
88311,643980,Ironbound
88312,670290,Real Pool 3D - Poolians
88313,767400,弹炸人2222
88314,773570,Log Challenge


In [21]:
games_to_add = games2[~games2['game_id'].isin(games['game_id'])]
games_to_add.head()

,game_id,name
88310,761140,Lost Summoner Kitty
88311,643980,Ironbound
88312,670290,Real Pool 3D - Poolians
88313,767400,弹炸人2222
88314,773570,Log Challenge


In [51]:
df_games = pd.DataFrame({'game_id':list(games['game_id']) + list(games2['game_id']),
'name':list(games['name']) + list(games2['name'])})
df_games.drop_duplicates(inplace=True)
df_games.drop_duplicates(subset='game_id', inplace=True)
df_games.drop_duplicates(subset='name', inplace=True)
df_games.dropna(inplace=True)
df_games.head()

,game_id,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [55]:
df_games.to_csv('../data/csv/games.csv')

In [2]:
games_detail = pd.read_json('../data/output_steam_games.json',lines = True)
games_detail.dropna(how='all', inplace=True)
games_detail.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


## EDA

In [3]:
games_detail.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [3]:
games_detail = games_detail[[
    'id',
    #'app_name', 
    #'title', 
    'price', 
    'publisher', 
    'release_date',
    #'reviews_url', 
    'early_access', 
    'url', 
    'developer'
    #'tags', --> nested
    #'genres', --> nested
    #'specs', --> nested
    ]]

In [5]:
games_detail.shape

(32135, 7)

## ETL
#### Handling null and missing values

In [8]:
games_detail['price'] = games_detail['price'].replace(to_replace=r'[^0-9]', value=0, regex=True) #replace all the registers in price that no contains numeric values with 0
games_detail['price'] = pd.to_numeric(games_detail['price'], errors='coerce')#transform values to numeric
games_detail['release_date'] = pd.to_datetime(games_detail['release_date'], errors='coerce', format='mixed', dayfirst=True)
games_detail['publisher'].replace(np.nan, 'No info', inplace=True)
games_detail['developer'].replace(np.nan, 'No info', inplace=True)
games_detail.dropna(subset=['id'], inplace=True)
games_detail.drop_duplicates(subset='id', inplace=True)
games_detail['early_access'] = games_detail['early_access'].astype('bool')
games_detail['id'] = games_detail['id'].astype('int64')
games_detail.shape

(32132, 7)

In [12]:
games_detail.rename(columns={'id':'game_id'}, inplace=True)
games_detail.head()

,game_id,price,publisher,release_date,early_access,url,developer
88310,761140,4.99,Kotoshiro,2018-01-04,False,http://store.steampowered.com/app/761140/Lost_...,Kotoshiro
88311,643980,0.00,"Making Fun, Inc.",2018-01-04,False,http://store.steampowered.com/app/643980/Ironb...,Secret Level SRL
88312,670290,0.00,Poolians.com,2017-07-24,False,http://store.steampowered.com/app/670290/Real_...,Poolians.com
88313,767400,0.99,彼岸领域,2017-12-07,False,http://store.steampowered.com/app/767400/2222/,彼岸领域
88314,773570,2.99,No info,NaT,False,http://store.steampowered.com/app/773570/Log_C...,No info


In [13]:
games_detail.to_csv('../data/csv/games_detail.csv')

In [17]:
user_items = {
    'user_id':[],
    'game_id':[],
    'hours_played':[]
}
for i, row in df_items[['user_id','items']].iterrows():
    user_id = row['user_id']
    for item in row['items']:
        game_id = item['item_id']
        hours = item['playtime_forever']
        user_items['user_id'].append(user_id)
        user_items['game_id'].append(game_id)
        user_items['hours_played'].append(hours)

In [23]:
user_items_df = pd.DataFrame(user_items)
user_items_df.to_csv('../data/csv/user_games.csv')

In [16]:
user_games = pd.read_csv('../data/csv/user_games.csv')[['user_id', 'game_id','hours_played']]
user_games = user_games[user_games['game_id'].isin(list(games['game_id'])) & user_games['user_id'].isin(list(user['user_id']))]
user_games.to_csv('../data/csv/user_gamesv2.csv')

In [18]:
pd.read_csv('../data/csv/user_gamesv2.csv')

,Unnamed: 0,user_id,game_id,hours_played
0,0,76561197970982479,10,6
1,1,76561197970982479,20,0
2,2,76561197970982479,30,7
3,3,76561197970982479,40,0
4,4,76561197970982479,50,0
...,...,...,...,...
5130603,5153204,76561198329548331,346330,0
5130604,5153205,76561198329548331,373330,0
5130605,5153206,76561198329548331,388490,3
5130606,5153207,76561198329548331,521570,4


In [18]:
reviews_root = '../data/australian_user_reviews.json'
reviews_rows = []
with open(reviews_root, encoding='MacRoman') as f:
    for line in f.readlines():
        reviews_rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(reviews_rows)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [19]:
dicc_users_reviews = {
    'user_id':[],
    'funny':[],
    'posted':[],
    'last_edited':[],
    'item_id':[],
    'helpful':[],
    'recommend':[],
    'review':[]
}
for i, row in df_reviews[['user_id','reviews']].iterrows():
    user_id = row['user_id']
    for review in row['reviews']:
        dicc_users_reviews['user_id'].append(user_id)
        dicc_users_reviews['funny'].append(review['funny'])
        dicc_users_reviews['posted'].append(review['posted'])
        dicc_users_reviews['last_edited'].append(review['last_edited'])
        dicc_users_reviews['item_id'].append(review['item_id'])
        dicc_users_reviews['helpful'].append(review['helpful'])
        dicc_users_reviews['recommend'].append(review['recommend'])
        dicc_users_reviews['review'].append(review['review'])

df_users_reviews = pd.DataFrame(dicc_users_reviews)
df_users_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [20]:
df_users_reviews.shape

(59305, 8)

In [21]:
user_reviews = df_users_reviews.copy()
user_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [22]:
from datetime import datetime
dates = []
for element in user_reviews['posted']:
    try:
        try:
            element = element.replace('Posted ', '').replace('.', '')
            
            date_obj = str(datetime.strptime(element, '%B %d, %Y'))[:10]
            dates.append(date_obj)
        except ValueError:
            element = element + ', 2023'
            date_obj = str(datetime.strptime(element, '%B %d, %Y'))[:10]
            dates.append(date_obj)
    except ValueError:
        dates.append('2020-02-29')


In [23]:
dates_series = pd.Series(dates)
user_reviews['posted'] = dates_series
user_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [17]:
user_reviews.columns

Index(['user_id', 'funny', 'posted', 'last_edited', 'game_id', 'helpful',
       'recommend', 'review', 'dates'],
      dtype='object')

In [24]:
user_reviews.rename(columns={'item_id':'game_id'}, inplace=True)
user_reviews.set_index('user_id').to_csv('../data/csv/user_reviews.csv')